# ISEL - CPS 
## Semestre 2021/22
# <br>
###  <center> Trabalho Prático 2</center>
# </br>

Trabalho realizado por:
* Fábio Dias, número <b> 42921 </b>
* Jorge Silva, número <b> 44615 </b>




A figura representa o esquema típico do processo de envio e recepção de informação usando modulações
digitais.

![fig_trab](esquemaCPS.png)

O 2º trabalho foca os blocos Codificação/descodificação do sinal e os blocos Deteção e Correção de erros. O canal é modelado pelo modelo Binary Symmetric Channel ([BSC](https://en.wikipedia.org/wiki/Binary_symmetric_channel)) 

# Codificação
Construa uma função Python que dado um Numpy array (com dimensão $N$) com valores inteiros, retorne um Numpy array (com dimensão $N.R\times 1$) com os mesmos valores convertidos para binário.
Esta função ainda deve receber como parâmetros de entrada o número de bits ($R$) a usar na conversão de cada inteiro.


In [16]:
import numpy as np
import matplotlib.pyplot as plt

def Codific(arr_signal_quantized,R):
    
    arr_binary = np.zeros(len(arr_signal_quantized)*R)
    binaryIndex = 0
    
    for signalValue in arr_signal_quantized:
        binaryValue = bin(signalValue).replace('0b', '')
        binaryValue = (R - len(binaryValue)) * "0" + binaryValue
        
        for index in range(R):
            arr_binary[binaryIndex] = binaryValue[index]
            binaryIndex += 1
        
    return arr_binary

print("Signal: ")
print([2, 2, 1, 2, 1, 2, 0])
print("Binary: ")
print([1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0])
print("Outcome (Binary): ")
print(Codific([2, 2, 1, 2, 1, 2, 0], 2))

Signal: 
[2, 2, 1, 2, 1, 2, 0]
Binary: 
[1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0]
Outcome (Binary): 
[1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0.]


Construa uma função que dado um Numpy array com valores os bits, faça a conversão para inteiros (considerando R bits).

In [17]:
def Descodific(arr_binary,R):
    
    arr_signal = np.zeros(int(len(arr_binary) / R))
    signalIndex = 0
    
    for binaryValue in range(0, int(len(arr_binary) / R)):
        
        binaryList = int(''.join(map(str, arr_binary[binaryValue * R : binaryValue * R + R])), 2)
        
        arr_signal[binaryValue] = binaryList
        
    return arr_signal

print("Signal: ")
print([2, 2, 1, 2, 1, 2, 0])
print("Binary: ")
print([1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0])
print("Outcome (Signal): ")
print(Descodific([1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0], 2))

Signal: 
[2, 2, 1, 2, 1, 2, 0]
Binary: 
[1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0]
Outcome (Signal): 
[2. 2. 1. 2. 1. 2. 0.]


Com base nas funções já implementadas, faça a codificação e descodificação PCM uniforme do sinal de áudio previamente gravado.
Verifique qual a SNR obtida e ouça o sinal descodificado. 
Use para o efeito R=3, 5 e 8.

In [18]:
from scipy.io import wavfile as wave

audio = wave.read("Audios/AudioDeUmMinuto.wav")[1][:,0]

#Copia do primeiro trabalho - Inicio

#Quantificador
def Quantific(R,Vmax,Qtype):
    
    if Qtype != "midrise" and Qtype != "midtread":

        print("Tipo de Quantificador inserido não válido")

        return
    
    delta = (2*Vmax) / 2**R
    
    if Qtype == "midrise":
        
        Iq = np.arange(-Vmax + delta, Vmax + delta, delta)

        Vq = np.arange(-Vmax + (delta/2), Vmax + (delta/2), delta)
    
    else:
        
        Iq = np.arange(-Vmax + (delta/2), Vmax, delta)
        
        Vq = np.arange(-Vmax + delta, Vmax + delta, delta)

    return Vq, Iq


#Quantificacao
def Quantificador(Rx, Vq, Iq):
    
    xq = np.zeros(len(Rx))
    iq = np.zeros(len(Rx), dtype="int")

    indexQuantificado = 0
    
    for valorDoSinal in Rx:
    
        indexIntervalos = 0
        
        for intervaloDecisao in Iq:

            if valorDoSinal <= intervaloDecisao:
                xq[indexQuantificado] = Vq[indexIntervalos]
                iq[indexQuantificado] = indexIntervalos
                indexQuantificado += 1
                break
                
            if indexIntervalos >= len(Iq) - 1:
                xq[indexQuantificado] = Vq[indexIntervalos]
                iq[indexQuantificado] = indexIntervalos
                indexQuantificado += 1
                break
            
            indexIntervalos += 1
                
    return xq, iq


#Desquantificador
def Desquantificador(Vq, iq):
    xq = np.zeros(len(iq))
    
    for valor in range(len(iq)):
        xq[valor] = Vq[iq[valor]]
        
    return xq


#SNR Prático
def Measure_SNRp(x,y):
    
    erro = x - y
    
    potenciaSinal = np.sum(x**2.0)/len(x)
    potenciaErro = np.sum(erro**2.0)/len(erro)
    
    SNRp = 10*np.log10(potenciaSinal / potenciaErro)
    
    # resposta
    return SNRp


#SNR Teórico
def Measure_SNRt(R,Vmax,P):
    
    SNRt = 6.02*R + 10*np.log10((3*P)/Vmax**2.0)
    
    #resposta
    return SNRt

##Copia do primeiro trabalho - Fim

Vmax = np.max(np.abs(audio))

for R in [3, 5, 8]:
    Vq, Iq = Quantific(R, Vmax, "midrise")
    xq, iq = Quantificador(audio, Vq, Iq)
    sinalCodificado = Codific(iq, R)
    sinalDescodificado = Descodific(sinalCodificado.astype(int), R)
    sinalDes = Desquantificador(Vq, sinalDescodificado.astype(int))

    potenciaSinal = np.sum(xq**2.0)/len(xq) #Isto está correcto?

    print("R: " + str(R))
    print("SNR Teórico: " + str(Measure_SNRt(R, Vmax, potenciaSinal)))
    print("SNR Prático: " + str(Measure_SNRp(audio, sinalDes)))
    print()

R: 3
SNR Teórico: 9.875106061812886
SNR Prático: 6.850051463632915

R: 5
SNR Teórico: 21.124500043702504
SNR Prático: 19.42233695641331

R: 8
SNR Teórico: 39.126081079372454
SNR Prático: 38.73646485050661



Faça o mesmo teste mas agora com a imagem fornecida.
comente os resultados obtidos.

In [19]:
from PIL import Image
im = Image.open("lena_gray.tif")

print(im)

np.array(im)

#imageBits = np.array(im[])
#print(len(imageBits))
#Resposta

<PIL.TiffImagePlugin.TiffImageFile image mode=L size=512x512 at 0x289C6F67160>


array([[162, 162, 162, ..., 170, 155, 128],
       [162, 162, 162, ..., 170, 155, 128],
       [162, 162, 162, ..., 170, 155, 128],
       ...,
       [ 43,  43,  50, ..., 104, 100,  98],
       [ 44,  44,  55, ..., 104, 105, 108],
       [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8)

# Deteção e Correção de erros

Construa uma função que implemente o Hamming(8,4). Utilize a seguinte matriz geradora:

In [20]:
G=np.array([[1,0,0,0,1,1,0,1],[0,1,0,0,1,0,1,1],[0,0,1,0,0,1,1,1],[0,0,0,1,1,1,1,0]])
G

array([[1, 0, 0, 0, 1, 1, 0, 1],
       [0, 1, 0, 0, 1, 0, 1, 1],
       [0, 0, 1, 0, 0, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 0]])

In [21]:
import numpy as np
import matplotlib.pyplot as plt

def Hamming_8_4(arr_bits):

    c = np.zeros(0)
    mAux = np.zeros(0)
    
    if(len(arr_bits) % 4 != 0):
        mAux = np.zeros(len(arr_bits) + (4 - (len(arr_bits) % 4)))
        mAux[0 : len(arr_bits)] = arr_bits[0 : len(arr_bits)]
        mAux[len(arr_bits) + 1 : len(mAux) + 1] = 0
        c = np.zeros(len(mAux) * 2)
    else:
        c = np.zeros(len(arr_bits) * 2)
        mAux = arr_bits
    
    for bits in range(0, int(len(mAux)/4)):
        m = mAux[bits * 4 : 4 * bits + 4]
        cAux = np.dot(m,G)
        c[bits * 8 : 8 * bits + 8] = np.mod(cAux, 2)
    
    return c

Construa uma função que dado um Numpy array M bits, para cada 8 bits detecte e corriga possíveis erros, sendo o resultado 4 bits de mensagem.


In [22]:
def Detetor(arr_binary):
    
    arr_bits = np.zeros(int(len(arr_binary)/2))
    
    Gt = np.matrix.transpose(G)
    
    for binarySegment in range(int(len(arr_binary)/8)):
        
        binary = np.dot(arr_binary[8 * binarySegment : 8 * binarySegment + 8], Gt)
        binary = np.mod(binary, 2)
        
        if(sum(binary) != 0):
            
            canItBeFixed = False
            
            for validationValuesInGt in range(len(Gt)):
                
                if((binary == Gt[validationValuesInGt]).all()):
                    arr_binary[binarySegment * 8 + validationValuesInGt] = 1 if(arr_binary[binarySegment * 8 + validationValuesInGt] == 0) else 0
                    arr_bits[binarySegment * 4 : binarySegment * 4 + 4] = arr_binary[binarySegment * 8 : binarySegment * 8 + 4]
                    canItBeFixed = True
                    break
                    
            if(not canItBeFixed):
                arr_bits[binarySegment * 4 : binarySegment * 4 + 4] = arr_binary[binarySegment * 8 : binarySegment * 8 + 4]
                    
        else:
            arr_bits[binarySegment * 4 : binarySegment * 4 + 4] = arr_binary[binarySegment * 8 : binarySegment * 8 + 4]
        
    
    return arr_bits

Faça um conjunto de testes para verificar o correto funcionamento das funções.

In [23]:
######### Hamming_8_4

print("---------------------------")
print("Hamming_8_4")
print("---------------------------")
print()

#Exemplo 1
m1 = [1, 0, 0, 0]
c1 = Hamming_8_4(m1)

#Exemplo 2
m2 = [1, 0, 0, 0, 0, 0, 1, 0]
c2 = Hamming_8_4(m2)

#Exemplo 3
m3 = [1, 0, 0, 0, 0, 0, 1, 0, 0, 1]
c3 = Hamming_8_4(m3)

#Exemplo 4
m4 = [0]
c4 = Hamming_8_4(m4)


print("Mensagem Original 1: " + str(m1))
print("Expectavel - Codificacao Hamming 1: " + str([1, 1, 0, 1]))
print("Mensagem Codificada Hamming 1: " + str(c1))

print()

print("Mensagem Original 2: " + str(m2))
print("Expectavel - Codificacao Hamming 2: " + str([1, 1, 0, 1]) + " & " + str([0, 1, 1, 1]))
print("Mensagem Codificada Hamming 2: " + str(c2))

print()

print("Mensagem Original 3: " + str(m3))
print("Expectavel - Codificacao Hamming 3: " + str([1, 1, 0, 1]) + " & " + str([0, 1, 1, 1]) + " & " + str([1, 0, 1, 1]))
print("Mensagem Codificada Hamming 3: " + str(c3))

print()

print("Mensagem Original 4: " + str(m4))
print("Expectavel - Codificacao Hamming 4: " + str([0, 0, 0, 0]))
print("Mensagem Codificada Hamming 4: " + str(c4))

print()
print()


######### Detetor

print("---------------------------")
print("Detetor")
print("---------------------------")
print()

#Certo e validado
c1 = [1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1]
m1 = Detetor(c1)

print("Palavras de Código Correctas e Validadas")
print()
print("Palavras de Código : " + str(c1))
print("Mensagem Expectável:    " + str([1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]))
print("Mensagem Descodificada: " + str(m1))
print()


#Incorrecto e Corrigido
c2 = [1, 0, 0, 0, 0, 1, 0, 1]
mE2 = [1, 0, 0, 0]
m2 = Detetor(c2)

print("Palavra de Código Incorrecta (Bit de Paridade 1), Detectada e Corrigida")
print()
print("Palavra de Código Correcto: " + str([1, 0, 0, 0, 1, 1, 0, 1]))
print("Palavra de Código Errado:   " + str([1, 0, 0, 0, 0, 1, 0, 1]))
print("Mensagem Expectável:    " + str(mE2))
print("Mensagem Descodificada: " + str(m2))
print()


#Incorrecto e Corrigido x2
c3 = [1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0]
mE3 = [1, 0, 0, 0, 1, 0, 0, 0]
m3 = Detetor(c3)

print("Duas Palavras de Código Incorrectas (Bit de Paridade 1 e Bit de Paridade 4), Detectadas e Corrigidas")
print()
print("Palavra de Código Correcto: " + str([1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1]))
print("Palavra de Código Errado:   " + str([1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0]))
print("Mensagem Expectável:    " + str(mE3))
print("Mensagem Descodificada: " + str(m3))
print()

#Palavra de Código Incorrecta Diferente, Detectada e Corrigida
c4 = [1, 0, 1, 0, 1, 1, 1, 0]
mE4 = [1, 0, 1, 0]
m4 = Detetor(c4)

print("Palavra de Código Incorrecta (Bit de Paridade 2) Diferente, Detectada e Corrigida")
print()
print("Palavra de Código Correcto: " + str([1, 0, 1, 0, 1, 0, 1, 0]))
print("Palavra de Código Errado:   " + str([1, 0, 1, 0, 1, 1, 1, 0]))
print("Mensagem Expectável:    " + str(mE4))
print("Mensagem Descodificada: " + str(m4))
print()

#Palavra de Código Incorrecta Diferente, Detectada e Corrigida
c5 = [1, 1, 1, 1, 0, 0, 0, 1]
mE5 = [1, 1, 1, 0]
m5 = Detetor(c5)

print("Palavra de Código Incorrecta (Bit de Mensagem 4) Diferente, Detectada e Corrigida")
print()
print("Palavra de Código Correcto: " + str([1, 1, 1, 0, 0, 0, 0, 1]))
print("Palavra de Código Errado:   " + str([1, 1, 1, 1, 0, 0, 0, 1]))
print("Mensagem Expectável:    " + str(mE5))
print("Mensagem Descodificada: " + str(m5))
print()
print()


##### Testes com Hamming_8_4 e Detetor

print("---------------------------")
print("Hamming_8_4 & Detetor")
print("---------------------------")
print()

#Exemplo 1
mensagemTeste1 = [1, 0, 0, 0]
palavraDeCodigoTeste1 = Hamming_8_4(mensagemTeste1)
mensagemDetetadaTeste1 = Detetor(palavraDeCodigoTeste1)

print("Exemplo 1")
print("Mensagem Teste: " + str(mensagemTeste1))
print("Palavra de Código após Hamming: " + str(palavraDeCodigoTeste1))
print("Mensagem após Detetor: " + str(mensagemDetetadaTeste1))
print()


#Exemplo 2

mensagemTeste2 = [1, 0, 0, 0]
palavraDeCodigoTeste2 = Hamming_8_4(mensagemTeste2)
palavraDeCodigoErradaTeste2 = [1, 0, 1, 0, 1, 1, 0, 1]
mensagemDetetadaTeste2 = Detetor(palavraDeCodigoErradaTeste2)

print("Exemplo 2")
print("Mensagem Teste: " + str(mensagemTeste2))
print("Palavra de Código após Hamming: " + str(palavraDeCodigoTeste2))
print("Mudança no terceiro bit da Palavra de Código: " + str([1, 0, 1, 0, 1, 1, 0, 1]))
print("Mensagem após Detetor: " + str(mensagemDetetadaTeste2))
print()

---------------------------
Hamming_8_4
---------------------------

Mensagem Original 1: [1, 0, 0, 0]
Expectavel - Codificacao Hamming 1: [1, 1, 0, 1]
Mensagem Codificada Hamming 1: [1. 0. 0. 0. 1. 1. 0. 1.]

Mensagem Original 2: [1, 0, 0, 0, 0, 0, 1, 0]
Expectavel - Codificacao Hamming 2: [1, 1, 0, 1] & [0, 1, 1, 1]
Mensagem Codificada Hamming 2: [1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1.]

Mensagem Original 3: [1, 0, 0, 0, 0, 0, 1, 0, 0, 1]
Expectavel - Codificacao Hamming 3: [1, 1, 0, 1] & [0, 1, 1, 1] & [1, 0, 1, 1]
Mensagem Codificada Hamming 3: [1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1.]

Mensagem Original 4: [0]
Expectavel - Codificacao Hamming 4: [0, 0, 0, 0]
Mensagem Codificada Hamming 4: [0. 0. 0. 0. 0. 0. 0. 0.]


---------------------------
Detetor
---------------------------

Palavras de Código Correctas e Validadas

Palavras de Código : [1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1]
Mensagem Expectável:    [1,

Construa uma função que meça a BER entre o array de bits enviado e o array de bits recebido.

In [24]:
def BER_pratico(arr_bin_tx, arr_bin_rx):
    
    BER_pratico = np.sum(arr_bin_tx != arr_bin_rx) / len(arr_bin_tx)
    
    return BER_pratico

Com as duas funções realizadas e admitindo que os restantes blocos do emissor, canal e receptor possam ser simulados pela expressão $y=1*np.logical\_xor(x,np.random.binomial(1, BER_t,len(x)))$, onde $BER_t$ contém o valor pretendido para o bit error rate.

Meça o BER antes e após a correcção de erros, para diferentes valores de $BER_t$.

In [25]:
#x - array de bits a entrar no canal
#y - array de bits a sair do canal

def EmissorCanalReceptor(x, BERt):
    
    y = 1 * np.logical_xor( x, np.random.binomial( 1, BERt, len(x) ) )

    return y

##Testes##

#Exemplo 1
mensagem1 = np.array([1, 0, 0, 0])
palavraDeCodigo1 = Hamming_8_4(mensagem1)
palavraAposCanal1 = EmissorCanalReceptor(palavraDeCodigo1, 0)
mensagemDetetada1 = Detetor(palavraAposCanal1)
ber1 = BER_pratico(mensagem1, mensagemDetetada1)

print("Exemplo 1")
print()

print("Mensagem: " + str(mensagem1))
print("Palavra de Código (após Hamming_8_4): " + str(palavraDeCodigo1))
print("Palavra após Simulação do Canal, com o BERt de " + str(0) + " :" + str(palavraAposCanal1))
print("Mensagem após Detetor: " + str(mensagemDetetada1))
print()
print("BER: " + str(ber1))
print()
print("------------------")
print()

#Exemplo 2
mensagem2 = np.array([1, 0, 0, 0])
palavraDeCodigo2 = Hamming_8_4(mensagem2)
palavraAposCanal2 = EmissorCanalReceptor(palavraDeCodigo2, 0.5)
mensagemDetetada2 = Detetor(palavraAposCanal2)
ber2 = BER_pratico(mensagem2, mensagemDetetada2)

print("Exemplo 2")
print()

print("Mensagem: " + str(mensagem2))
print("Palavra de Código (após Hamming_8_4): " + str(palavraDeCodigo2))
print("Palavra após Simulação do Canal, com o BERt de " + str(0.5) + " :" + str(palavraAposCanal2))
print("Mensagem após Detetor: " + str(mensagemDetetada2))
print()
print("BER: " + str(ber2))
print()
print("------------------")
print()

#Exemplo 3
mensagem3 = np.array([1, 0, 0, 0, 1, 0, 0, 0])
palavraDeCodigo3 = Hamming_8_4(mensagem3)
palavraAposCanal3 = EmissorCanalReceptor(palavraDeCodigo3, 1)
mensagemDetetada3 = Detetor(palavraAposCanal3)
ber3 = BER_pratico(mensagem3, mensagemDetetada3)

print("Exemplo 3")
print()

print("Mensagem: " + str(mensagem3))
print("Palavra de Código (após Hamming_8_4): " + str(palavraDeCodigo3))
print("Palavra após Simulação do Canal, com o BERt de " + str(1) + " :" + str(palavraAposCanal3))
print("Mensagem após Detetor: " + str(mensagemDetetada3))
print()
print("BER: " + str(ber3))
print()
print("------------------")
print()

Exemplo 1

Mensagem: [1 0 0 0]
Palavra de Código (após Hamming_8_4): [1. 0. 0. 0. 1. 1. 0. 1.]
Palavra após Simulação do Canal, com o BERt de 0 :[1 0 0 0 1 1 0 1]
Mensagem após Detetor: [1. 0. 0. 0.]

BER: 0.0

------------------

Exemplo 2

Mensagem: [1 0 0 0]
Palavra de Código (após Hamming_8_4): [1. 0. 0. 0. 1. 1. 0. 1.]
Palavra após Simulação do Canal, com o BERt de 0.5 :[0 1 1 1 0 1 0 0]
Mensagem após Detetor: [0. 1. 1. 1.]

BER: 1.0

------------------

Exemplo 3

Mensagem: [1 0 0 0 1 0 0 0]
Palavra de Código (após Hamming_8_4): [1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1.]
Palavra após Simulação do Canal, com o BERt de 1 :[0 1 1 1 0 0 1 0 0 1 1 1 0 0 1 0]
Mensagem após Detetor: [0. 1. 1. 1. 0. 1. 1. 1.]

BER: 1.0

------------------



Considere o sinal de áudio previamente gravado. Meça SNR na recepção, o BER antes e após a correcção de erros, para diferentes valores de  𝐵𝐸𝑅𝑡 . (considera-se que o ruído é a diferença entre o sinal à entrada do transmissor e o sinal à saída do receptor); Comente os resultados obtidos.

In [26]:
Vmax = np.max(np.abs(audio))

for R in [3, 5, 8]:
    
    
    print("R : " + str(R))
    
    Vq, Iq = Quantific(R, Vmax, "midrise")
    xq, iq = Quantificador(audio, Vq, Iq)

    potenciaSinal = np.sum(xq**2.0)/len(xq) #Isto está correcto?
    
    sinalCodificado = Codific(iq, R)
    sinalPalavraDeCodigo = Hamming_8_4(sinalCodificado)
    
    for ruido in [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2]:
        sinalPalavraDeCodigoAposRuido = EmissorCanalReceptor(sinalPalavraDeCodigo, ruido)
        sinalDetetado = Detetor(sinalPalavraDeCodigoAposRuido)
        sinalDescodificado = Descodific(sinalCodificado.astype(int), R)
        sinalDes = Desquantificador(Vq, sinalDescodificado.astype(int))

        ber = BER_pratico(np.array(sinalPalavraDeCodigo), np.array(sinalPalavraDeCodigoAposRuido))
        berLinha = BER_pratico(np.array(sinalCodificado), np.array(sinalDetetado))
        
        print("Ruido: " + str(ruido))
        print("BER: " + str(ber))
        print("BER': " + str(berLinha))
        print()
        
    print("SNR Teórico: " + str(Measure_SNRt(R, Vmax, potenciaSinal)))
    print("SNR Prático: " + str(Measure_SNRp(audio, sinalDes)))
    print()

R : 3
Ruido: 0.0001
BER: 0.0
BER': 0.0

Ruido: 0.0005
BER: 0.0
BER': 0.0

Ruido: 0.001
BER: 2.2675736961451248e-05
BER': 2.2675736961451248e-05

Ruido: 0.005
BER: 0.00020408163265306123
BER': 0.00020408163265306123

Ruido: 0.01
BER: 0.0007558578987150416
BER': 0.0008163265306122449

Ruido: 0.05
BER: 0.015850340136054422
BER': 0.0163567649281935

Ruido: 0.1
BER: 0.05643990929705216
BER': 0.056424792139077855

Ruido: 0.2
BER: 0.17600907029478458
BER': 0.17621315192743764

SNR Teórico: 9.875106061812886
SNR Prático: 6.850051463632915

R : 5
Ruido: 0.0001
BER: 0.0
BER': 0.0

Ruido: 0.0005
BER: 4.535147392290249e-06
BER': 4.535147392290249e-06

Ruido: 0.001
BER: 0.0
BER': 0.0

Ruido: 0.005
BER: 0.00015873015873015873
BER': 0.00016780045351473923

Ruido: 0.01
BER: 0.0006848072562358276
BER': 0.0007573696145124716

Ruido: 0.05
BER: 0.015405895691609977
BER': 0.015541950113378685

Ruido: 0.1
BER: 0.0565124716553288
BER': 0.05597732426303855

Ruido: 0.2
BER: 0.17418140589569162
BER': 0.17385941

Considere a imagem fornecida e meça a SNR e o BER com e sem correção de erros.
Visualize a imagem no receptor e comente os resultados.

In [27]:
from PIL import Image

img = np.asarray(Image.open('lena_gray.tif'))

arr_img = img.flatten()

print(arr_img)

Vmax = np.max(np.abs(arr_img))

for R in [3, 5, 8]:
    
    
    print("R : " + str(R))
    
    Vq, Iq = Quantific(R, Vmax, "midrise")
    xq, iq = Quantificador(arr_img, Vq, Iq)

    potenciaSinal = np.sum(xq**2.0)/len(xq) #Isto está correcto?
    
    sinalCodificado = Codific(iq, R)
    sinalPalavraDeCodigo = Hamming_8_4(sinalCodificado)
    
    for ruido in [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2]:
        sinalPalavraDeCodigoAposRuido = EmissorCanalReceptor(sinalPalavraDeCodigo, ruido)
        sinalDetetado = Detetor(sinalPalavraDeCodigoAposRuido)
        sinalDescodificado = Descodific(sinalCodificado.astype(int), R)
        sinalDes = Desquantificador(Vq, sinalDescodificado.astype(int))

        ber = BER_pratico(np.array(sinalPalavraDeCodigo), np.array(sinalPalavraDeCodigoAposRuido))
        berLinha = BER_pratico(np.array(sinalCodificado), np.array(sinalDetetado))
        
        print("Ruido: " + str(ruido))
        print("BER: " + str(ber))
        print("BER': " + str(berLinha))
        print()
        
    print("SNR Teórico: " + str(Measure_SNRt(R, Vmax, potenciaSinal)))
    print("SNR Prático: " + str(Measure_SNRp(arr_img, sinalDes)))
    print()

[162 162 162 ... 104 105 108]
R : 3
Ruido: 0.0001
BER: 0.0
BER': 0.0

Ruido: 0.0005
BER: 1.2715657552083333e-06
BER': 1.2715657552083333e-06

Ruido: 0.001
BER: 1.0172526041666666e-05
BER': 1.3987223307291666e-05

Ruido: 0.005
BER: 0.00021107991536458334
BER': 0.00020345052083333334

Ruido: 0.01
BER: 0.0006968180338541666
BER': 0.00070953369140625

Ruido: 0.05
BER: 0.01573944091796875
BER': 0.015837351481119793

Ruido: 0.1
BER: 0.056437174479166664
BER': 0.05650456746419271

Ruido: 0.2
BER: 0.17525736490885416
BER': 0.17513529459635416

SNR Teórico: 17.69136727594082
SNR Prático: 17.865473777988036

R : 5
Ruido: 0.0001
BER: 0.0
BER': 0.0

Ruido: 0.0005
BER: 0.0
BER': 0.0

Ruido: 0.001
BER: 6.866455078125e-06
BER': 9.1552734375e-06

Ruido: 0.005
BER: 0.0001556396484375
BER': 0.000154876708984375

Ruido: 0.01
BER: 0.0006805419921875
BER': 0.000675201416015625

Ruido: 0.05
BER: 0.015908050537109374
BER': 0.015853118896484376

Ruido: 0.1
BER: 0.05619125366210938
BER': 0.05619888305664063

R